# Full Tracker: Download, Process, and Upload Data
This notebook demonstrates the full pipeline for handling raw data:
1. Download data from a cloud bucket.
2. Process the data (e.g., align videos, run detection, and tracking).
3. Upload the processed data back to the cloud bucket.

In [ ]:
# Import Required Libraries
import os
import uuid
from pathlib import Path
from Desktop.labeling_data.data_structure.Dima_collab.collab_data.file_utils import expand_path, get_project_root
from Desktop.labeling_data.data_structure.Dima_collab.collab_data.gcs_utils import GCSClient

In [ ]:
# Setup Configuration

#doing it with full path because I'm too lazy for actual directory management 
CREDENTIALS_PATH = "/Users/inesaitsahalia/Desktop/labeling_data/data_structure/Dima_collab/collab_data/api-keys/collab-data-463313-c340ad86b28e.json"
PROJECT_ID = "collab-data-463313"
BUCKET_NAME = "fieldwork_data"  # Update with your bucket name
LOCAL_DOWNLOAD_DIR = Path("/path/to/local/download")  # Update with your local download directory
LOCAL_PROCESSED_DIR = Path("/path/to/local/processed")  # Update with your local processed directory

In [ ]:
# Connect to Google Cloud Storage
gcs_client = GCSClient(
    project_id=PROJECT_ID,
    credentials_path=CREDENTIALS_PATH,
    )

# Verify connection
print("Available buckets:", gcs_client.list_buckets())

In [ ]:
# Download Data from Cloud Bucket
CLOUD_PREFIX = "your-cloud-prefix"  # Update with your cloud prefix
LOCAL_DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

for blob in gcs_client.glob(f"{BUCKET_NAME}/{CLOUD_PREFIX}/**"):
    local_path = LOCAL_DOWNLOAD_DIR / Path(blob).name
    gcs_client.download_file(blob, str(local_path))

print("Downloaded files:", list(LOCAL_DOWNLOAD_DIR.iterdir()))

In [ ]:
# Process the Data
print("Processing data...")
for file in LOCAL_DOWNLOAD_DIR.iterdir():
    if file.suffix == ".csq":
        print(f"Converting CSQ thermal file: {file}")
        # Convert CSQ thermal files
        converted_file = LOCAL_PROCESSED_DIR / f"{file.stem}.mp4"
        os.system(f"python pipeline_code/thermal_processing.py --input {file} --output {converted_file}")

    if file.suffix == ".mp4":
        print(f"Aligning and processing video: {file}")
        # Step 1: Align videos
        aligned_rgb_dir = LOCAL_PROCESSED_DIR / "aligned_rgb"
        aligned_thm_dir = LOCAL_PROCESSED_DIR / "aligned_thm"
        aligned_rgb_dir.mkdir(parents=True, exist_ok=True)
        aligned_thm_dir.mkdir(parents=True, exist_ok=True)
        os.system(f"python pipeline_code_out_of_the_box/scripts/align_videos_manually.py --rgb_video_path {file} --thermal_video_path {file} --output_dir_rgb {aligned_rgb_dir} --output_dir_thm {aligned_thm_dir} --frame_size 640,480")

        # Step 2: Run detection and tracking
        print(f"Running detection and tracking on: {file}")
        os.system(f"python pipeline_code/local_model_tracking.py --target_root_dir {LOCAL_PROCESSED_DIR} --session_root_dir {file.stem} --camera_type thermal --camera_number 1")

In [ ]:
# Upload Processed Data to Cloud Bucket
CLOUD_PROCESSED_PREFIX = "your-cloud-processed-prefix"  # Update with your processed data prefix
for file in LOCAL_PROCESSED_DIR.iterdir():
    cloud_path = f"{BUCKET_NAME}/{CLOUD_PROCESSED_PREFIX}/{file.name}"
    gcs_client.upload_file(str(file), cloud_path)

print("Uploaded processed files:", list(LOCAL_PROCESSED_DIR.iterdir()))